# 📊 Funding Rounds Data Collector
This notebook fetches data from CryptoRank APIs and saves them as CSV files.

In [1]:
import requests
import pandas as pd
import json
from pathlib import Path

# Ensure data folder exists
Path('../data').mkdir(parents=True, exist_ok=True)

## 🔹 Total Funding Overview

In [2]:
import requests
import pandas as pd

url = "https://api.cryptorank.io/v0/funding-rounds-widgets/total-funding-overview?locale=en"
response = requests.get(url)
data = response.json()

flat_data = {
    "fundingRoundsTotal": data["fundingRoundsTotal"],
    "roundsGrowth": data["fundingRoundsTotalChange"],  # ← already in %
    "totalInvestments": data["totalInvestments"],
    "volumeGrowth": data["totalInvestmentsChange"],    # ← already in %
    "investmentFocus_id": data["investmentsFocus"]["id"],
    "investmentFocus_name": data["investmentsFocus"]["name"],
    "investmentFocus_description": data["investmentsFocus"]["description"],
    "averageRoundSize": f"${data['avgRoundSize']['raise_from']:,} - ${data['avgRoundSize']['raise_to']:,}",
    "investmentStage": data["investmentStage"]
}

df_overview = pd.DataFrame([flat_data])
df_overview.to_csv("../data/total_funding_overview.csv", index=False)
df_overview


,fundingRoundsTotal,roundsGrowth,totalInvestments,volumeGrowth,investmentFocus_id,investmentFocus_name,investmentFocus_description,averageRoundSize,investmentStage
0,128,-21.95,4247155500,19.89,224,Artificial Intelligence (AI),Projects using AI-technology at the core of th...,"$3,000,000 - $10,000,000",SEED


## 🔹 Hot Rounds

In [3]:
import requests
import pandas as pd

url = "https://api.cryptorank.io/v0/funding-rounds-widgets/hot-rounds/"
response = requests.get(url)
data = response.json()

# Filter out hidden rounds and build a structured list
merged_data = {}
for item in data:
    if item.get("isHidden"):
        continue

    key = item["coinName"]
    if key not in merged_data:
        merged_data[key] = {
            "coinKey": item["coinKey"],
            "coinName": item["coinName"],
            "raise": round(item["raise"] / 1_000_000, 2),  # e.g., 140M
            "coinImage": item["coinImage"],
            "funds": []
        }

    existing_logos = {f["logo"] for f in merged_data[key]["funds"]}
    for fund in item.get("funds", []):
        if fund["logo"] not in existing_logos:
            merged_data[key]["funds"].append({
                "slug": fund["slug"],
                "logo": fund["logo"]
            })
            existing_logos.add(fund["logo"])

# Save to CSV with funds as JSON string
df_hot_rounds = pd.DataFrame([
    {
        "coinKey": v["coinKey"],
        "coinName": v["coinName"],
        "raise": v["raise"],
        "coinImage": v["coinImage"],
        "funds": v["funds"]
    }
    for v in merged_data.values()
])

df_hot_rounds.to_csv("../data/hot_rounds.csv", index=False)
df_hot_rounds.head()


,coinKey,coinName,raise,coinImage,funds
0,walrus-protocol,Walrus,140.0,https://img.cryptorank.io/coins/60x60.walrus17...,"[{'slug': 'standard-crypto', 'logo': 'https://..."
1,chronicle-labs,Chronicle,12.0,https://img.cryptorank.io/coins/60x60.chronicl...,"[{'slug': 'strobe-ventures', 'logo': 'https://..."
2,moonpay,MoonPay,200.0,https://img.cryptorank.io/coins/60x60.moon_pay...,"[{'slug': 'galaxy-digital', 'logo': 'https://i..."


## 🔹 Fundraising Digest

In [4]:
import requests
import pandas as pd
import json

url = "https://api.cryptorank.io/v0/funding-rounds-widgets/fundraising-digest?locale=en"
response = requests.get(url)
data = response.json()

records = []
for item in data:
    records.append({
        "title": item["title"],
        "publishDate": item["publishDate"],
        "url": item["url"],
        "type": item["type"],
        "subtype": item.get("subtype") or "",  
        "tags": [tag["name"] for tag in item.get("tags", [])]
    })

df_digest = pd.DataFrame(records)
df_digest.to_csv("../data/fundraising_digest.csv", index=False)


In [5]:

import requests
import pandas as pd

# Define API endpoint
url = "https://api.cryptorank.io/v0/funding-rounds-v2/"

# Prepare the payload and headers
payload = {
    "limit": 1000,  # Limit the results
    "offset": 0,
}
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)
data = response.json()

# Extract and normalize the data
rounds = pd.json_normalize(data['data'])

# Save to CSV for visualization in Django
rounds.to_csv("../data/cryptorank_funding_rounds.csv", index=False)
rounds.head()


,icon,key,name,symbol,date,raise,publicSalesRaise,totalRaise,stage,country,...,valuation,funds,hasFundingRounds,createdAt,isHidden,category.name,category.key,country.key,country.name,category
0,https://img.cryptorank.io/coins/60x60.echooo16...,echooo,Echooo,None,2023-07-17T00:00:00.000Z,15000000.0,NaN,15000000.0,Undisclosed,NaN,...,100000000.0,"[{'name': 'A&T Capital', 'key': 'at-capital', ...",False,2023-07-19T09:08:02.728Z,False,Blockchain Service,blockchain-service,NaN,NaN,NaN
1,https://img.cryptorank.io/coins/60x60.clockwor...,clockwork,Clockwork,None,2022-08-30T00:00:00.000Z,4000000.0,NaN,4000000.0,SEED,NaN,...,NaN,"[{'name': 'Multicoin Capital', 'key': 'multico...",False,2022-08-31T06:26:01.149Z,False,Blockchain Infrastructure,blockchain-infrastructure,US,United States,NaN
2,https://img.cryptorank.io/coins/60x60.tokenpro...,tokenproof,Tokenproof,None,2022-08-30T00:00:00.000Z,5000000.0,NaN,5000000.0,SEED,NaN,...,NaN,"[{'name': 'Penny Jar Capital', 'key': 'penny-j...",False,2022-08-31T08:03:41.461Z,False,NFT,non-fungible-tokens-nft,US,United States,NaN
3,https://img.cryptorank.io/coins/60x60.ether_ma...,ethermail,EtherMail,EMT,2022-08-11T00:00:00.000Z,3000000.0,60000.0,7060000.0,SEED,NaN,...,NaN,"[{'name': 'Fabric Ventures', 'key': 'fabric-ve...",True,2022-08-31T10:50:24.972Z,False,Blockchain Service,blockchain-service,LI,Liechtenstein,NaN
4,https://img.cryptorank.io/coins/60x60.unlockd1...,unlockd,Unlockd,UNLK,2022-08-18T00:00:00.000Z,4440000.0,NaN,4440000.0,SEED,NaN,...,NaN,"[{'name': 'Blockchain Capital', 'key': 'blockc...",False,2022-08-31T13:17:37.878Z,False,DeFi,defi,ES,Spain,NaN


In [6]:
import requests
import pandas as pd
from pathlib import Path
from datetime import datetime

# Prepare the payload and headers
url = "https://api.cryptorank.io/v0/funding-rounds-v2/"
payload = {
    "limit": 1000,
    "offset": 0
}
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
response.raise_for_status()

raw = response.json()["data"]

# ===================== Main Flattened Records =====================
main_records = []
for item in raw:
    main_records.append({
        "coinKey": item.get("key"),
        "coinName": item.get("name"),
        "coinImage": item.get("icon"),
        "raise": item.get("raise"),
        "stage": item.get("stage"),
        "date": item.get("date"),
        "valuation": item.get("valuation"),
        "followersCount": item.get("followersCount"),
        "twitterScore": item.get("twitterScore"),
        "status": item.get("status"),
        "isHidden": item.get("isHidden"),
        "country_name": item["country"]["name"] if item.get("country") else None,
        "country_key": item["country"]["key"] if item.get("country") else None,
        "category_name": item["category"]["name"] if item.get("category") else None,
        "category_key": item["category"]["key"] if item.get("category") else None
    })

# Sort by date and keep only the latest 1000
def parse_date(x):
    try:
        return datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ")
    except:
        return datetime.min

main_records = sorted(main_records, key=lambda x: parse_date(x["date"]), reverse=True)[:1000]

# ===================== Flatten Funds =====================
funds_records = []
for item in raw:
    for fund in item.get("funds", []):
        funds_records.append({
            "coinKey": item["key"],
            "coinName": item["name"],
            "raise": item["raise"],
            "coinImage": item["icon"],
            "fund_name": fund.get("name"),
            "fund_key": fund.get("key"),
            "fund_image": fund.get("image"),
            "fund_tier": fund.get("tier"),
            "fund_type": fund.get("type"),
            "fund_category": fund["category"]["name"] if fund.get("category") else None,
            "fund_category_slug": fund["category"]["slug"] if fund.get("category") else None,
            "fund_totalInvestments": fund.get("totalInvestments"),
        })

# ===================== Flatten Top Followers =====================
follower_records = []
for item in raw:
    for follower in item.get("topFollowers", []):
        follower_records.append({
            "coinKey": item["key"],
            "coinName": item["name"],
            "follower_id": follower.get("id"),
            "follower_name": follower.get("name"),
            "follower_username": follower.get("username"),
            "follower_image": follower.get("image"),
            "follower_twitterScore": follower.get("twitterScore"),
            "follower_followersCount": follower.get("followersCount"),
            "relatedEntity_fund": follower.get("relatedEntity", {}).get("fund") if follower.get("relatedEntity") else None,
            "relatedEntity_coin": follower.get("relatedEntity", {}).get("coin") if follower.get("relatedEntity") else None
        })

# Save to CSV
pd.DataFrame(main_records).to_csv("../data/funding_main.csv", index=False)
pd.DataFrame(funds_records).to_csv("../data/funding_funds.csv", index=False)
pd.DataFrame(follower_records).to_csv("../data/funding_followers.csv", index=False)

print("CSV overwritten at", Path("../data/funding_main.csv").resolve())
print("Fetched records:", len(main_records))
print("Latest round:", main_records[0] if main_records else "None")


CSV overwritten at C:\Users\BRHN\Desktop\Due-Diligence\DueDiligence\data\funding_main.csv
Fetched records: 1000
Latest round: {'coinKey': 'redotpay', 'coinName': 'RedotPay', 'coinImage': 'https://img.cryptorank.io/coins/60x60.redot_pay1741937484272.png', 'raise': 40000000, 'stage': 'SERIES A', 'date': '2025-03-14T00:00:00.000Z', 'valuation': None, 'followersCount': 85708, 'twitterScore': 649, 'status': 'not traded', 'isHidden': False, 'country_name': 'Hong Kong', 'country_key': 'HK', 'category_name': 'Blockchain Service', 'category_key': 'blockchain-service'}
